In [52]:
#import required libraries
import pandas as pd
import requests
import numpy as np
import re

In [2]:
#Task1: All alphabetical characters in all columns should be capitalized except (DOB, POBox, POCityStateZip).
data = pd.read_csv('inputDB.csv')

for i in data.columns: 
    if i not in ['DOB','POBox','POCityStateZip']:
        data[i] = data[i].str.upper()

In [3]:
#Task2: Address data should be compatible to the standard

data = pd.read_csv('inputDB.csv')

length = len(data)
is_address = np.zeros(length, dtype='str')

is_address = data['Address'].str.split(' ')
is_address = is_address.apply(lambda x:True if (x[0].isdigit() and len(x[0])>=3 and len(x[0]) <=5)  else False)
is_address = pd.DataFrame(is_address)

zip_addr = list(zip(data['Address'],is_address['Address']))
zip_addr = pd.Series(zip_addr)
zip_addr = zip_addr.apply(lambda x: str(x[0]).upper() if x[1] == True else str(x[0]).upper())
data['Address'] = zip_addr

In [78]:
#Task3: The State column should contain the correct two character US state code.

#Create a dictionary for states {'two letter names' : 'full names'}
states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AS': 'American Samoa',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'GU': 'Guam',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MP': 'Northern Mariana Islands',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NA': 'National',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'PR': 'Puerto Rico',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VI': 'Virgin Islands',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

data = pd.read_csv('inputDB.csv')

#-----------Split the dictionary for key, values and full items----------
states_key = states.keys()
states_value = list(states.values())
states_value = [x.upper() for x in states_value] #convert values to UpperCase
states_item = states.items()
length = len(data)
#------------------------------------------------------------------------

#-----------Declare arrays for each column that we are gonna check the State value in---------
state_from_state = np.zeros(length,dtype='str')
state_from_POCityStateZip = np.zeros(length,dtype='str')
state_from_zip = np.zeros(length,dtype='str')
#--------------------------------------------------------------------

#-----------looking for State from State column itself--------------
c1 = data['State'].isin(states_key)
c2 = data['State'].isin(states_value)

state_from_state = list(zip(data['State'],c2,c1))
state_from_state = pd.Series(state_from_state).apply(lambda z :[shortcut for shortcut, state_name in states_item if str(state_name).upper() == z[0]][0] if z[1] else z[0] if z[2] else '') 
state_from_state = pd.DataFrame(state_from_state)
#--------------------------------------------------------------------

#-----------Looking for State from ZIP column------------------------
state_from_zip = data['ZIP'].str.split(' ')
state_from_zip = state_from_zip.apply(lambda x: x[0] if len(x)==2 else x[1] if len(x)==3 else '')
c1 = state_from_zip.isin(states_key)
c2 = state_from_zip.isin(states_value)
state_from_zip = list(zip(state_from_zip,c2,c1))
state_from_zip = pd.Series(state_from_zip).apply(lambda z :[shortcut for shortcut, state_name in states_item if str(state_name).upper() == z[0]][0] if z[1] else z[0] if z[2] else '') 
state_from_zip = pd.DataFrame(state_from_zip)
#--------------------------------------------------------------------

#------------Looking for State from POCityStateZip column------------
state_from_POCityStateZip = data['POCityStateZip'].str.split(',')
state_from_POCityStateZip = state_from_POCityStateZip.apply(lambda x: x[-1] if str(x)!='nan' else '' )
state_from_POCityStateZip = state_from_POCityStateZip.apply(lambda x: x.split(' ')[-(len(x.split(' '))-1)]if str(x)!='nan' else '' )
c1 = state_from_POCityStateZip.isin(states_key)
c2 = state_from_POCityStateZip.isin(states_value)
state_from_POCityStateZip = list(zip(state_from_POCityStateZip,c2,c1))
state_from_POCityStateZip = pd.Series(state_from_POCityStateZip).apply(lambda z :[shortcut for shortcut, state_name in states_item if str(state_name).upper() == z[0]][0] if z[1] else z[0] if z[2] else '') 
state_from_POCityStateZip = pd.DataFrame(state_from_POCityStateZip)
#--------------------------------------------------------------------

#combine the true values with zip function and edit the State column in the original dataframe
combinedStates = list(zip(state_from_state[0],state_from_POCityStateZip[0],state_from_zip[0]))
combinedStates = pd.DataFrame(combinedStates)
combinedStates = combinedStates.apply(lambda x: x[0] if str(x[0])!='' else x[1] if str(x[1])!= '' else str(x[2]),axis=1)
data['State'] = combinedStates


In [77]:
#Task4: City column should contain real city names.
data = pd.read_csv('inputDB.csv')

length = len(data)
city_from_city = np.zeros(length, dtype='str')

for i in data.columns: 
    if i  in ['City']:#,'POBox','POCityStateZip']:
        data[i] = data[i].str.upper()

is_city = data['City'].notnull()
city_from_city = np.where(is_city, data['City'], city_from_city)

city_from_POCityStateZip = np.zeros(length, dtype='str')
city_from_POCityStateZip = data['POCityStateZip'].str.split(',')
city_from_POCityStateZip = city_from_POCityStateZip.apply(lambda x: x[0] if str(x) != 'nan' else '')

city_from_city = pd.DataFrame(city_from_city)
city_from_POCityStateZip = pd.DataFrame(city_from_POCityStateZip)

true_city = list(zip(city_from_city[0],city_from_POCityStateZip['POCityStateZip']))
true_city = pd.DataFrame(true_city)
true_city = true_city.apply(lambda x: str(x[0]).upper() if x[0]!='' else str(x[1]).upper() ,axis=1)
data['City'] = true_city


In [69]:
#Task5: ZIP column should be formatted as a 5 digit value.
data = pd.read_csv('inputDB.csv')
states_key = states.keys()
states_item = states.items()
states_value = pd.Series(states.values()).str.upper()
length = len(data)

zip_from_zip = np.zeros(length, dtype='str')
zip_from_POCityStateZip = np.zeros(length, dtype='str')


state_from_zip = data['ZIP'].str.split(' ')
#Convert all characters to UpperCase if the length of the word is 2
zip_from_zip = state_from_zip.apply(lambda x: str(x[1]).upper() if len(x)==2 else
                                  str(x[2]).upper() if len(x)==3 else
                                  str(x[0]).upper() if len(x)==1 else '' )

zip_from_zip = zip_from_zip.apply(lambda x: x if str(x).isdigit() and len(str(x))==5 else
                                '' if str(x).isdigit() and len(str(x))!=5 else x)

zip_from_zip = zip_from_zip.apply(lambda x : re.split('\D',x))

zip_from_zip = zip_from_zip.apply(lambda x : str(x[3]) if len(x)==4 else
                                str(x[0]) + str(x[1]) if len(x)==2 and str(x[0]).isdigit() else
                                str(x[1]) if len(x)==2 and str(x[0])=='' else
                                '' if  len(x)==2 else
                                str(x))

zip_from_zip = zip_from_zip.apply(lambda x:  x[2:7] if len(x)==9 else '' if len(x)==4 else str(x) )

zip_from_zip = pd.DataFrame(zip_from_zip)

state_from_POCityStateZip = data['POCityStateZip'].str.split(',')
state_from_POCityStateZip = state_from_POCityStateZip.apply(lambda x: x[-1] if str(x)!='nan' else '' )
state_from_POCityStateZip

zip_from_POCityStateZip = state_from_POCityStateZip.apply(lambda x: x.split(' ')[(len(x.split(' '))-1)]if str(x)!='nan' else '' )

zip_from_POCityStateZip = zip_from_POCityStateZip.apply(lambda x :str(x.split('.')[-1:]).upper()  if '.' in str(x) 
                                                      else (str(x).upper()) if str(x).isdigit() else '')

zip_from_POCityStateZip = pd.DataFrame(zip_from_POCityStateZip)

combinedZIP = list(zip(zip_from_zip['ZIP'],zip_from_POCityStateZip['POCityStateZip']))
combinedZIP = pd.DataFrame(combinedZIP)
combinedZIP = combinedZIP.apply(lambda x: x[0] if x[0]!='' else x[1], axis=1)
combinedZIP = combinedZIP.apply( lambda x : x if len(x)==5 else '')
combinedZIP = pd.DataFrame(combinedZIP)
data['ZIP'] = combinedZIP

In [76]:
#Task6: SSN column should contain an 8-10 digit value

data = pd.read_csv('inputDB.csv')

#filter null values
data['SSN'] = data['SSN'].apply(lambda x: '' if str(x)== 'nan' else x)

length = len(data)
ssn = np.zeros(length, dtype='str')
ssn = data['SSN'].apply(lambda x: str(x).split('-'))
ssn = ssn.apply(lambda x: '' if str(x)=='' else  (x[0]+x[1]+x[2]) if len(x)== 3 else x[0])
ssn = ssn.apply(lambda x : '' if str(x)=='' else x if (len(x) >=8 and len(x)<=10 )else '')
ssn = pd.DataFrame(ssn)
data['SSN'] = ssn

In [82]:
data = pd.read_csv('inputDB.csv')

for i in data.columns: 
    if i not in ['DOB','POBox','POCityStateZip']:
        data[i] = data[i].str.upper()

data['SSN'] = ssn
data['ZIP'] = combinedZIP
data['City'] = true_city
data['State'] = combinedStates
data['Address'] = zip_addr

data.to_csv("test.csv")

In [86]:
url = "http://gorilla.bigdama.tu-berlin.de:8001/DataCleaningEvaluator/"
dataset = open("test.csv", "r").read()
r = requests.post(url, data={"value": dataset})
if r.status_code != 200:
    print("There is an error! The error code:", r.status_code)
print(r.text)

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))